<a href="https://colab.research.google.com/github/jkff-mv/stable-diffusion-text2img-example/blob/master/stable_diffusion_text2img_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusionで画像を生成する

## Overview

Google Colaboratory上で [Stable Diffusion](https://github.com/CompVis/stable-diffusion) のtext2imgを動かすサンプルです。テキストからそれっぽい画像を生成します。

## Requirement

[Hugging Face](https://huggingface.co/) へのユーザ登録が必要です。登録が完了したら以下の作業を実施ください。

* [こちらのページ](https://huggingface.co/CompVis/stable-diffusion-v1-4) にアクセスし、Stable Diffusionのモデルの使用に同意する。
* [こちらのページ](https://huggingface.co/settings/tokens) にアクセスし、Hugging Faceのアクセストークンを発行する。

また以下のコードを実行し、GPUが割り当てられていることを確認ください。割り当てられていない場合「編集」→「ノートブックの設定」から割り当てが可能です。使用量の上限に引っかかって割り当てができない場合は時間をおいて再度お試しください。

In [ ]:
!nvidia-smi -L

## Install

必要な依存関係をインストールします。以下のコードを実行ください。

In [ ]:
!pip install --upgrade diffusers==0.2.4 transformers==4.* scipy==1.* ftfy==6.*

## Setup

Stable Diffusionのモデルのダウンロードやパイプラインの構築を行います。予め発行したHugging Faceのアクセストークンを `access_token` に設定し、以下のコードを実行ください。

In [ ]:
from diffusers import StableDiffusionPipeline

#@title Settings
access_token = "your-access-token"  #@param {type:"string"}

pipe = StableDiffusionPipeline.from_pretrained(
  "CompVis/stable-diffusion-v1-4",
  use_auth_token=access_token
).to("cuda")

## Run

画像を生成します。生成したい画像の内容を `prompt` に記述し以下のコードを実行ください。

`number_of_images` は生成する画像数です。複数の画像を生成したい場合は変更ください。

In [ ]:
from IPython.display import display_png, Image
from torch import autocast

#@title Parameters
prompt = "a description of the image"  #@param {type:"string"}
number_of_images = 1  #@param [1, 2, 3, 4, 5] {type:"raw"}

for i in range(number_of_images):
  with autocast("cuda"):
    image = pipe(prompt)["sample"][0]

  file_name = f"image-{i:03}.png"
  image.save(file_name)

  display_png(Image(file_name))

## Run and Export Images to Google Drive

画像を生成してGoogleドライブに保存します。

Googleドライブをマウントします。以下のコードを実行ください。

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

画像を生成します。生成したい画像の内容を `prompt` に記述し以下のコードを実行ください。

`number_of_images` は生成する画像数です。必要に応じて調整ください。なお数が多すぎるとGoogle Colabのリソース制限に引っかかり、しばらくGPUが利用できなくなる恐れがあります。

`drive_directory` は保存先ドライブのディレクトリです。例えばマイドライブの `output` 配下に画像を保存する場合 `/content/drive/MyDrive/output` のように記載ください。

In [ ]:
import datetime
import os
from torch import autocast

#@title Parameters
prompt = "a description of the image"  #@param {type:"string"}
number_of_images = 10  #@param {type:"slider", min:1, max:100, step:1}
drive_directory = "/content/drive/MyDrive/output"  #@param {type:"string"}

os.makedirs(drive_directory, exist_ok=True)

yyyymmddhhmmss = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

for i in range(number_of_images):
  with autocast("cuda"):
    image = pipe(prompt)["sample"][0]

  file_path = os.path.join(drive_directory, f"image-{yyyymmddhhmmss}-{i:03}.png")  
  image.save(file_path)

ドライブのマウントを解除します。以下のコードを実行ください。

In [ ]:
drive.flush_and_unmount()